In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tqdm import tqdm
import copy

In [2]:
z = ZipFile("../caltech-cs155-2020.zip")

dfs = {text_file.filename: pd.read_csv(z.open(text_file.filename))
       for text_file in z.infolist()
       if text_file.filename.endswith('.csv')}
dfs.keys()

dict_keys(['sample_submission.csv', 'test.csv', 'train.csv'])

In [3]:
df_train = dfs['train.csv']
df_test = dfs['test.csv']
df_train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [6]:
# Remove all rows with NaN to have it cleaner
array_train = df_train.to_numpy()
array_no_NaN = array_train[~np.isnan(array_train).any(axis = 1)]
array_no_NaN

array([[1.00000e+00, 3.84280e+03, 3.84340e+03, ..., 1.00000e+00,
        1.30000e+01, 0.00000e+00],
       [2.00000e+00, 3.84400e+03, 3.84430e+03, ..., 4.00000e+00,
        9.00000e+00, 0.00000e+00],
       [3.00000e+00, 3.84380e+03, 3.84340e+03, ..., 2.00000e+00,
        1.10000e+01, 1.00000e+00],
       ...,
       [5.92374e+05, 4.10940e+03, 4.10980e+03, ..., 1.00000e+01,
        7.00000e+00, 1.00000e+00],
       [5.92375e+05, 4.11020e+03, 4.11030e+03, ..., 7.00000e+00,
        7.00000e+00, 1.00000e+00],
       [5.92376e+05, 4.10940e+03, 4.11050e+03, ..., 7.00000e+00,
        5.00000e+00, 0.00000e+00]])

In [7]:
# Normalize feature vectors
norm_train = np.empty_like(array_train)

for i in range(28):
    col_max = max(array_no_NaN[:, i])
    col_min = min(array_no_NaN[:, i])
    norm_train[:, i] = (array_train[:, i] - col_min) / (col_max - col_min)

In [8]:
array_train = df_train.to_numpy()
# Make a copy of the column, so we don't change the original
vector_3 = copy.deepcopy(norm_train[:,3])
vector_4 = copy.deepcopy(norm_train[:,4])

# Train the first imputer
vector_3 = np.reshape(vector_3, (-1,1))
imp_v3 = IterativeImputer(max_iter=100, random_state=0)
imp_v3.fit(vector_3)
vector_3_trafo = imp.transform(vector_3)
vector_3_trafo = vector_3_trafo.flatten()

# Train the second imputer
vector_4 = np.reshape(vector_4, (-1,1))
imp_v4 = IterativeImputer(max_iter=100, random_state=0)
imp_v4.fit(vector_4)
vector_4_trafo = imp.transform(vector_4)
vector_4_trafo = vector_4_trafo.flatten()

# Put back into the normalized array
transform_train = copy.deepcopy(norm_train)
transform_train[:,3] = vector_3_trafo
transform_train[:,4] = vector_4_trafo

NameError: name 'imp' is not defined